In [1]:
# Cell 1: Imports and Setup
from osgeo import gdal
import atlite
import geopandas as gpd
import pypsa
import matplotlib.pyplot as plt
import pandas as pd
import cartopy.crs as ccrs
import p_H2_aux as aux
from functions import CRF
import numpy as np
import logging
import time
import xarray as xr
from scipy.constants import physical_constants

# Logging setup
logging.basicConfig(level=logging.ERROR)


In [14]:
def hydropower_potential(eta,flowrate,head):
    '''
    Calculate hydropower potential in Megawatts

    Parameters
    ----------
    eta : float
        Efficiency of the hydropower plant.
    flowrate : float
        Flowrate calculated with runoff multiplied by the hydro-basin area, in cubic meters per hour.
    head : float
        Height difference at the hydropower site, in meters.

    Returns
    -------
    float
        Hydropower potential in Megawatts (MW).
    '''
    rho = 997 # kg/m3; Density of water
    g = physical_constants['standard acceleration of gravity'][0] # m/s2; Based on the CODATA constants 2018
    Q = flowrate / 3600 # transform flowrate per h into flowrate per second
    return (eta * rho * g * Q * head) / (1000 * 1000) # MW

def hydropower_potential_with_capacity(flowrate, head, capacity, eta):
    '''
    Calculate the hydropower potential considering the capacity limit

    Parameters
    ----------
    flowrate : float
        Flowrate calculated with runoff multiplied by the hydro-basin area, in cubic meters per hour.
    head : float
        Height difference at the hydropower site, in meters.
    capacity : float
        Maximum hydropower capacity in Megawatts (MW).
    eta : float
        Efficiency of the hydropower plant.

    Returns
    -------
    xarray DataArray
        Capacity factor, which is the limited potential divided by the capacity.
    '''
    potential = hydropower_potential(flowrate, head, eta)
    limited_potential = xr.where(potential > capacity, capacity, potential)
    capacity_factor = limited_potential / capacity
    return capacity_factor

In [15]:
def demand_schedule(quantity, transport_state, transport_excel_path,
                             weather_excel_path):
    '''
    calculates hourly hydrogen demand for truck shipment and pipeline transport.

    Parameters
    ----------
    quantity : float
        annual amount of hydrogen to transport in kilograms.
    transport_state : string
        state hydrogen is transported in, one of '500 bar', 'LH2', 'LOHC', or 'NH3'.
    transport_excel_path : string
        path to transport_parameters.xlsx file
    weather_excel_path : string
        path to transport_parameters.xlsx file
            
    Returns
    -------
    trucking_hourly_demand_schedule : pandas DataFrame
        hourly demand profile for hydrogen trucking.
    pipeline_hourly_demand_schedule : pandas DataFrame
        hourly demand profile for pipeline transport.
    '''
    transport_parameters = pd.read_excel(transport_excel_path,
                                         sheet_name = transport_state,
                                         index_col = 'Parameter'
                                         ).squeeze('columns')
    weather_parameters = pd.read_excel(weather_excel_path,
                                       index_col = 'Parameters',
                                       ).squeeze('columns')
    truck_capacity = transport_parameters['Net capacity (kg H2)']
    start_date = weather_parameters['Start date']
    end_date = weather_parameters['End date (not inclusive)']

    # Adjust capacity based on excess
    # schedule for trucking
    annual_deliveries = quantity/truck_capacity
    quantity_per_delivery = quantity/annual_deliveries
    index = pd.date_range(start_date, end_date, periods=annual_deliveries)
    trucking_demand_schedule = pd.DataFrame(quantity_per_delivery, index=index, columns = ['Demand'])
    trucking_hourly_demand_schedule = trucking_demand_schedule.resample('h').sum().fillna(0.)

    # schedule for pipeline
    index = pd.date_range(start_date, end_date, freq = 'h')
    pipeline_hourly_quantity = quantity/index.size
    pipeline_hourly_demand_schedule = pd.DataFrame(pipeline_hourly_quantity, index=index,  columns = ['Demand'])

    return trucking_hourly_demand_schedule, pipeline_hourly_demand_schedule

In [18]:
def optimize_hydrogen_plant(wind_potential, pv_potential, hydro_potential, times, demand_profile, 
                            wind_max_capacity, pv_max_capacity, hydro_max_capacity,
                            country_series, water_limit=None):
    '''
    Optimizes the size of green hydrogen plant components based on renewable potential, hydrogen demand, and country parameters. 

    Parameters
    ----------
    wind_potential : xarray DataArray
        1D dataarray of per-unit wind potential in hexagon.
    pv_potential : xarray DataArray
        1D dataarray of per-unit solar potential in hexagon.
    times : xarray DataArray
        1D dataarray with timestamps for wind and solar potential.
    demand_profile : pandas DataFrame
        hourly dataframe of hydrogen demand in kg.
    country_series : pandas Series
        interest rate and lifetime information.
    water_limit : float, optional
        annual limit on water available for electrolysis in hexagon, in cubic meters. Default is None.
    hydro_potential : xarray DataArray, optional
        1D dataarray of per-unit hydro potential in hexagon. Default is None.
    hydro_max_capacity : float, optional
        maximum hydro capacity in MW. Default is None.

    Returns
    -------
    lcoh : float
        levelized cost per kg hydrogen.
    wind_capacity: float
        optimal wind capacity in MW.
    solar_capacity: float
        optimal solar capacity in MW.
    hydro_capacity: float
        optimal hydro capacity in MW.
    electrolyzer_capacity: float
        optimal electrolyzer capacity in MW.
    battery_capacity: float
        optimal battery storage capacity in MW/MWh (1 hour batteries).
    h2_storage: float
        optimal hydrogen storage capacity in MWh.
    '''
   
    # if a water limit is given, check if hydrogen demand can be met
    if water_limit is not None:
        total_hydrogen_demand = demand_profile['Demand'].sum()
        water_constraint = total_hydrogen_demand <= water_limit * 111.57  # kg H2 per cubic meter of water
        if not water_constraint:
            print('Not enough water to meet hydrogen demand!')
            return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan

    # Set up network
    n = pypsa.Network(override_component_attrs=aux.create_override_components())
    n.set_snapshots(times)
    n.import_from_csv_folder("Parameters/Basic_H2_plant")
    n.add('Load', 'Hydrogen demand', bus='Hydrogen', p_set=demand_profile['Demand'] / 1000 * 39.4)

    # Send the weather data to the model
    n.generators_t.p_max_pu['Wind'] = wind_potential
    n.generators_t.p_max_pu['Solar'] = pv_potential
    n.generators_t.p_max_pu['Hydro'] = hydro_potential
    n.generators.loc['Hydro', 'p_nom_max'] = hydro_max_capacity 
    n.generators.loc['Wind', 'p_nom_max'] = wind_max_capacity *2
    n.generators.loc['Solar', 'p_nom_max'] = pv_max_capacity

    # Specify technology-specific and country-specific WACC and lifetime
    n.generators.loc['Hydro', 'capital_cost'] *= CRF(country_series['Hydro interest rate'], 
                                                    country_series['Hydro lifetime'])
    n.generators.loc['Wind', 'capital_cost'] *= CRF(country_series['Wind interest rate'], 
                                                    country_series['Wind lifetime (years)'])
    n.generators.loc['Solar', 'capital_cost'] *= CRF(country_series['Solar interest rate'], 
                                                     country_series['Solar lifetime (years)'])
    for item in [n.links, n.stores, n.storage_units]:
        item.capital_cost *= CRF(country_series['Plant interest rate'], 
                                 country_series['Plant lifetime (years)'])

    # Solve the model
    solver = 'gurobi'
    n.lopf(solver_name=solver,
           solver_options={'LogToConsole': 0, 'OutputFlag': 0},
           pyomo=False,
           extra_functionality=aux.extra_functionalities)

    # Output results
    total_cost = n.objective
    total_hydrogen_produced = n.loads_t.p_set.sum().iloc[0] / 39.4 * 1000  # in kg H2

    # Print detailed breakdown of costs
    print(f"Total cost (objective): {total_cost}")
    print(f"Total hydrogen produced: {total_hydrogen_produced} kg H2")
    
    # Breakdown of installed capacities and costs
    wind_capacity = n.generators.p_nom_opt['Wind']
    solar_capacity = n.generators.p_nom_opt['Solar']
    hydro_capacity = n.generators.p_nom_opt.get('Hydro', np.nan)
    electrolyzer_capacity = n.links.p_nom_opt['Electrolysis'] #* 1000
    battery_capacity = n.storage_units.p_nom_opt['Battery']
    h2_storage = n.stores.e_nom_opt['Compressed H2 Store']

    print(f"Optimal wind capacity: {wind_capacity} MW")
    print(f"Optimal solar capacity: {solar_capacity} MW")
    print(f"Optimal hydro capacity: {hydro_capacity} MW")
    print(f"Optimal electrolyzer capacity: {electrolyzer_capacity} MW")
    print(f"Optimal battery capacity: {battery_capacity} MW")
    print(f"Optimal hydrogen storage capacity: {h2_storage} MWh")
    
    # Calculate and print the Levelized Cost of Hydrogen (LCOH)
    lcoh = total_cost / total_hydrogen_produced  # cost per kg H2
    print(f"Levelized Cost of Hydrogen (LCOH): {lcoh} per kg H2")
    
    return lcoh, wind_capacity, solar_capacity, hydro_capacity, electrolyzer_capacity, battery_capacity, h2_storage


In [ ]:

def optimize_hydrogen_plant(wind_potential, pv_potential, hydro_potential, times, demand_profile, 
                            wind_max_capacity, pv_max_capacity, hydro_max_capacity,
                            country_series, water_limit=None):
    '''
    Optimizes the size of green hydrogen plant components based on renewable potential, hydrogen demand, and country parameters. 

    Parameters
    ----------
    wind_potential : xarray DataArray
        1D dataarray of per-unit wind potential in hexagon.
    pv_potential : xarray DataArray
        1D dataarray of per-unit solar potential in hexagon.
    times : xarray DataArray
        1D dataarray with timestamps for wind and solar potential.
    demand_profile : pandas DataFrame
        hourly dataframe of hydrogen demand in kg.
    country_series : pandas Series
        interest rate and lifetime information.
    water_limit : float, optional
        annual limit on water available for electrolysis in hexagon, in cubic meters. Default is None.
    hydro_potential : xarray DataArray, optional
        1D dataarray of per-unit hydro potential in hexagon. Default is None.
    hydro_max_capacity : float, optional
        maximum hydro capacity in MW. Default is None.

    Returns
    -------
    lcoh : float
        levelized cost per kg hydrogen.
    wind_capacity: float
        optimal wind capacity in MW.
    solar_capacity: float
        optimal solar capacity in MW.
    hydro_capacity: float
        optimal hydro capacity in MW.
    electrolyzer_capacity: float
        optimal electrolyzer capacity in MW.
    battery_capacity: float
        optimal battery storage capacity in MW/MWh (1 hour batteries).
    h2_storage: float
        optimal hydrogen storage capacity in MWh.
    '''
   
    # if a water limit is given, check if hydrogen demand can be met
    if water_limit != None:
        # total hydrogen demand in kg
        total_hydrogen_demand = demand_profile['Demand'].sum()
        # check if hydrogen demand can be met based on hexagon water availability
        water_constraint =  total_hydrogen_demand <= water_limit * 111.57 # kg H2 per cubic meter of water
        if water_constraint == False:
            print('Not enough water to meet hydrogen demand!')
            # return null values
            lcoh = np.nan
            wind_capacity = np.nan
            solar_capacity = np.nan
            hydro_capacity = np.nan
            electrolyzer_capacity = np.nan
            battery_capacity = np.nan
            h2_storage = np.nan
            return lcoh, wind_capacity, solar_capacity, hydro_capacity, electrolyzer_capacity, battery_capacity, h2_storage

    # Set up network
    # Import a generic network
    n = pypsa.Network(override_component_attrs=aux.create_override_components())

    # Set the time values for the network
    n.set_snapshots(times)

    # Import the design of the H2 plant into the network
    n.import_from_csv_folder("Parameters/Basic_H2_plant")

    # Import demand profile
    # Note: All flows are in MW or MWh, conversions for hydrogen done using HHVs. Hydrogen HHV = 39.4 MWh/t
    # hydrogen_demand = pd.read_excel(demand_path,index_col = 0) # Excel file in kg hydrogen, convert to MWh
    n.add('Load', 'Hydrogen demand', bus='Hydrogen', p_set=demand_profile['Demand'] / 1000 * 39.4)

    # Send the weather data to the model
    n.generators_t.p_max_pu['Wind'] = wind_potential
    n.generators_t.p_max_pu['Solar'] = pv_potential

    n.generators_t.p_max_pu['Hydro'] = hydro_potential
    # Specify maximum capacity based on land use
    n.generators.loc['Hydro', 'p_nom_max'] = hydro_max_capacity
    n.generators.loc['Wind', 'p_nom_max'] = wind_max_capacity
    n.generators.loc['Solar', 'p_nom_max'] = pv_max_capacity

    # Specify technology-specific and country-specific WACC and lifetime here
    n.generators.loc['Hydro', 'capital_cost'] *= CRF(country_series['Hydro interest rate'], 
                                                    country_series['Hydro lifetime'])
    n.generators.loc['Wind', 'capital_cost'] *= CRF(country_series['Wind interest rate'], 
                                                    country_series['Wind lifetime (years)'])
    n.generators.loc['Solar', 'capital_cost'] *= CRF(country_series['Solar interest rate'], 
                                                     country_series['Solar lifetime (years)'])

    for item in [n.links, n.stores, n.storage_units]:
        item.capital_cost = item.capital_cost * CRF(country_series['Plant interest rate'], country_series['Plant lifetime (years)'])

    # Solve the model
    solver = 'gurobi'
    # n.optimize(solver_name=solver,
    #        solver_options={'LogToConsole': 0, 'OutputFlag': 0},
    #        extra_functionality=aux.extra_functionalities)
    n.lopf(solver_name=solver,
           solver_options = {'LogToConsole':0, 'OutputFlag':0},
           pyomo=False,
           extra_functionality=aux.extra_functionalities,
           )

    # Output results
    lcoh = n.objective / (n.loads_t.p_set.sum().iloc[0] / 39.4 * 1000)  # convert back to kg H2
    wind_capacity = n.generators.p_nom_opt['Wind']
    solar_capacity = n.generators.p_nom_opt['Solar']
    hydro_capacity = n.generators.p_nom_opt.get('Hydro', np.nan)  # get hydro capacity if it exists
    electrolyzer_capacity = n.links.p_nom_opt['Electrolysis'] 
    battery_capacity = n.storage_units.p_nom_opt['Battery']
    h2_storage = n.stores.e_nom_opt['Compressed H2 Store']
    print(lcoh)
    return lcoh, wind_capacity, solar_capacity, hydro_capacity, electrolyzer_capacity, battery_capacity, h2_storage

In [7]:
weather_excel_path = "Parameters/weather_parameters.xlsx"
weather_parameters = pd.read_excel(weather_excel_path, index_col='Parameters').squeeze('columns')
weather_filename = weather_parameters['Filename']
hexagons = gpd.read_file('Resources/hex_transport.geojson')
cutout = atlite.Cutout('Cutouts_23/' + weather_filename + '.nc')
layout = cutout.uniform_layout()
# Added for hydropower
location_hydro = gpd.read_file('Data/hydropower_dams.gpkg')
location_hydro.rename(columns={'Latitude': 'lat', 'Longitude': 'lon'}, inplace=True)
location_hydro.rename(columns={'head_example':'head'}, inplace=True)

laos_hydrobasins = gpd.read_file('hydrobasins_lvl10/hybas_as_lev10_v1c.shp')
laos_hydrobasins['lat'] = location_hydro.geometry.y
laos_hydrobasins['lon'] = location_hydro.geometry.x

runoff = cutout.hydro(plants=location_hydro,
                      hydrobasins=laos_hydrobasins, 
                      per_unit=True)
eta = 0.75  # efficiency of hydropower plant

capacity_factor = xr.apply_ufunc(
    hydropower_potential_with_capacity,
    runoff,
    xr.DataArray(location_hydro['head'].values, dims=['plant']),
    xr.DataArray(location_hydro['capacity'].values, dims=['plant']),
    eta,
    vectorize=True,
    dask='parallelized',  # Dask for parallel computation
    output_dtypes=[float]
)

location_hydro['geometry'] = gpd.points_from_xy(location_hydro.lon, location_hydro.lat)

if 'index_left' in location_hydro.columns:
    location_hydro = location_hydro.rename(columns={'index_left': 'index_left_renamed'})
if 'index_right' in location_hydro.columns:
    location_hydro = location_hydro.rename(columns={'index_right': 'index_right_renamed'})
if 'index_left' in hexagons.columns:
    hexagons = hexagons.rename(columns={'index_left': 'index_left_renamed'})
if 'index_right' in hexagons.columns:
    hexagons = hexagons.rename(columns={'index_right': 'index_right_renamed'})

hydro_hex_mapping = gpd.sjoin(location_hydro, hexagons, how='left', predicate='within')
hydro_hex_mapping['plant_index'] = hydro_hex_mapping.index
num_hexagons = len(hexagons)
num_time_steps = len(capacity_factor.time)

hydro_profile = xr.DataArray(
    data=np.zeros((num_hexagons, num_time_steps)),
    dims=['hexagon', 'time'],
    coords={'hexagon': np.arange(num_hexagons), 'time': capacity_factor.time}
)

for hex_index in range(num_hexagons):
    plants_in_hex = hydro_hex_mapping[hydro_hex_mapping['index_right'] == hex_index]['plant_index'].tolist()
    if len(plants_in_hex) > 0:
        hex_capacity_factor = capacity_factor.sel(plant=plants_in_hex)
        plant_capacities = xr.DataArray(location_hydro.loc[plants_in_hex]['capacity'].values, dims=['plant'])

        weights = plant_capacities / plant_capacities.sum()
        weighted_avg_capacity_factor = (hex_capacity_factor * weights).sum(dim='plant')
        hydro_profile.loc[hex_index] = weighted_avg_capacity_factor

pv_profile = cutout.pv(
    panel='CSi',
    orientation='latitude_optimal',
    layout=layout,
    shapes=hexagons,
    per_unit=True
)
pv_profile = pv_profile.rename(dict(dim_0='hexagon'))

wind_profile = cutout.wind(
    turbine='Vestas_V80_2MW_gridstreamer',
    layout=layout,
    shapes=hexagons,
    per_unit=True
)
wind_profile = wind_profile.rename(dict(dim_0='hexagon'))


Determine upstream basins per plant: 81it [00:02, 27.26it/s]
c:\Users\lukas\anaconda3\envs\geoh2_master\Lib\site-packages\scipy\sparse\_base.py:730: RuntimeWarning: divide by zero encountered in divide
  recip = np.true_divide(1., other)


[########################################] | 100% Completed | 13.86 s


Shift and aggregate runoff by plant: 81it [00:04, 18.49it/s] 


[########################################] | 100% Completed | 86.24 s


c:\Users\lukas\anaconda3\envs\geoh2_master\Lib\site-packages\atlite\resource.py:72: FutureWarning: 'add_cutout_windspeed' for wind turbine
power curves will default to True in atlite relase v0.2.13.
  warnings.warn(msg, FutureWarning)


[########################################] | 100% Completed | 26.77 s


In [8]:
transport_excel_path = "Parameters/transport_parameters.xlsx"
country_excel_path = 'Parameters/country_parameters.xlsx'
country_parameters = pd.read_excel(country_excel_path, index_col='Country')
demand_excel_path = 'Parameters/demand_parameters.xlsx'
demand_parameters = pd.read_excel(demand_excel_path, index_col='Demand center').squeeze("columns")
demand_centers = demand_parameters.index

lcohs_trucking = np.zeros(len(pv_profile.hexagon))
solar_capacities = np.zeros(len(pv_profile.hexagon))
wind_capacities = np.zeros(len(pv_profile.hexagon))
hydro_capacities = np.zeros(len(pv_profile.hexagon))
electrolyzer_capacities = np.zeros(len(pv_profile.hexagon))
battery_capacities = np.zeros(len(pv_profile.hexagon))
h2_storages = np.zeros(len(pv_profile.hexagon))

In [27]:
# Cell 6: Optimization Loop for Specific Hexagon
specific_hexagon_index = 774 #563 #134 #719 # 100 # 719  # Replace with your specific hexagon index

for location in demand_centers:
    lcohs_trucking = np.zeros(len(pv_profile.hexagon))
    solar_capacities = np.zeros(len(pv_profile.hexagon))
    wind_capacities = np.zeros(len(pv_profile.hexagon))
    hydro_capacities = np.zeros(len(pv_profile.hexagon))
    electrolyzer_capacities = np.zeros(len(pv_profile.hexagon))
    battery_capacities = np.zeros(len(pv_profile.hexagon))
    h2_storages = np.zeros(len(pv_profile.hexagon))
    start = time.process_time()

    hexagon = specific_hexagon_index

    hydrogen_demand_trucking, _ = demand_schedule(
        demand_parameters.loc[location, 'Annual demand [kg/a]'],
        hexagons.loc[hexagon, f'{location} trucking state'],
        transport_excel_path,
        weather_excel_path
    )
    country_series = country_parameters.loc[hexagons.country[hexagon]]
    
    lcoh, wind_capacity, solar_capacity, hydro_capacity, electrolyzer_capacity, battery_capacity, h2_storage = \
        optimize_hydrogen_plant(
            wind_profile.sel(hexagon=hexagon),
            pv_profile.sel(hexagon=hexagon),
            hydro_profile.sel(hexagon=hexagon),
            wind_profile.time,
            hydrogen_demand_trucking,
            hexagons.loc[hexagon, 'theo_turbines'],
            hexagons.loc[hexagon, 'theo_pv'],
            hexagons.loc[hexagon, 'hydro'],
            country_series
        )

    lcohs_trucking[hexagon] = lcoh
    solar_capacities[hexagon] = solar_capacity
    wind_capacities[hexagon] = wind_capacity
    hydro_capacities[hexagon] = hydro_capacity
    electrolyzer_capacities[hexagon] = electrolyzer_capacity
    battery_capacities[hexagon] = battery_capacity
    h2_storages[hexagon] = h2_storage

    trucking_time = time.process_time() - start

    hexagons.at[hexagon, f'{location} trucking solar capacity'] = solar_capacities[hexagon]
    hexagons.at[hexagon, f'{location} trucking wind capacity'] = wind_capacities[hexagon]
    hexagons.at[hexagon, f'{location} trucking hydro capacity'] = hydro_capacities[hexagon]
    hexagons.at[hexagon, f'{location} trucking electrolyzer capacity'] = electrolyzer_capacities[hexagon]
    hexagons.at[hexagon, f'{location} trucking battery capacity'] = battery_capacities[hexagon]
    hexagons.at[hexagon, f'{location} trucking H2 storage capacity'] = h2_storages[hexagon]
    hexagons.at[hexagon, f'{location} trucking production cost'] = lcohs_trucking[hexagon]

    print(f"Trucking optimization for {location} and hexagon {hexagon} completed in {trucking_time} s")


C:\Users\lukas\AppData\Local\Temp\ipykernel_56756\897372266.py:39: FutureWarning: Non-integer 'periods' in pd.date_range, pd.timedelta_range, pd.period_range, and pd.interval_range are deprecated and will raise in a future version.
  index = pd.date_range(start_date, end_date, periods=annual_deliveries)
C:\Users\lukas\AppData\Local\Temp\ipykernel_56756\3564070593.py:79: DeprecatedWarning: lopf is deprecated as of 0.24 and will be removed in 1.0. Use linopy-based function ``n.optimize()`` instead. Migrate extra functionalities: https://pypsa.readthedocs.io/en/latest/examples/optimization-with-linopy-migrate-extra-functionalities.html.
  n.lopf(solver_name=solver,


Read LP format model from file C:\Users\lukas\AppData\Local\Temp\pypsa-problem-h2ut6zqm.lp
Reading time = 0.65 seconds
obj: 219600 rows, 96633 columns, 419240 nonzeros
Total cost (objective): 39050079.986231126
Total hydrogen produced: 3808999.9999999995 kg H2
Optimal wind capacity: 0.0 MW
Optimal solar capacity: 233.6400881952893 MW
Optimal hydro capacity: 0.0 MW
Optimal electrolyzer capacity: 111.86231705653888 MW
Optimal battery capacity: 0.0 MW
Optimal hydrogen storage capacity: 1180.1338006025765 MWh
Levelized Cost of Hydrogen (LCOH): 10.252055654038102 per kg H2
Trucking optimization for Southern and hexagon 774 completed in 17.65625 s


In [24]:
hexagons[hexagons["h3_index"] == '856590affffffff']#.loc[91, 'theo_pv']

,h3_index,n0,n1,n2,n3,n4,n5,waterbody_dist,waterway_dist,road_dist,...,Southern trucking state,Southern pipeline transport and conversion costs,geometry,Southern trucking solar capacity,Southern trucking wind capacity,Southern trucking hydro capacity,Southern trucking electrolyzer capacity,Southern trucking battery capacity,Southern trucking H2 storage capacity,Southern trucking production cost
774,856590affffffff,62,821,846,662,801,0,0.0,0.0,0.0,...,NH3,1.428329,"POLYGON ((105.81432 14.11739, 105.79214 14.210...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
specific_hexagon_index = 774#563

for location in demand_centers:
    

    # Pipeline optimization
    lcohs_pipeline = np.zeros(len(pv_profile.hexagon))
    solar_capacities = np.zeros(len(pv_profile.hexagon))
    wind_capacities = np.zeros(len(pv_profile.hexagon))
    hydro_capacities = np.zeros(len(pv_profile.hexagon))
    electrolyzer_capacities = np.zeros(len(pv_profile.hexagon))
    battery_capacities = np.zeros(len(pv_profile.hexagon))
    h2_storages = np.zeros(len(pv_profile.hexagon))
    start = time.process_time()

    hexagon = specific_hexagon_index

    _, hydrogen_demand_pipeline = demand_schedule(
        demand_parameters.loc[location,'Annual demand [kg/a]'],
        hexagons.loc[hexagon,f'{location} trucking state'],
        transport_excel_path,
        weather_excel_path)
    country_series = country_parameters.loc[hexagons.country[hexagon]]
    lcoh, wind_capacity, solar_capacity, hydro_capacity, electrolyzer_capacity, battery_capacity, h2_storage =\
        optimize_hydrogen_plant(wind_profile.sel(hexagon = hexagon),
                            pv_profile.sel(hexagon = hexagon),
                            hydro_profile.sel(hexagon = hexagon),
                            wind_profile.time,
                            hydrogen_demand_pipeline,
                            hexagons.loc[hexagon,'theo_turbines'],
                            hexagons.loc[hexagon,'theo_pv'],
                            hexagons.loc[hexagon,'hydro'],
                            country_series,
                            # water_limit = hexagons.loc[hexagon,'delta_water_m3']
                            )
    lcohs_trucking[hexagon] = lcoh
    solar_capacities[hexagon] = solar_capacity
    wind_capacities[hexagon] = wind_capacity
    hydro_capacities[hexagon] = hydro_capacity
    electrolyzer_capacities[hexagon] = electrolyzer_capacity
    battery_capacities[hexagon] = battery_capacity
    h2_storages[hexagon] = h2_storage

    pipeline_time = time.process_time() - start

    hexagons[f'{location} pipeline solar capacity'] = solar_capacities
    hexagons[f'{location} pipeline wind capacity'] = wind_capacities
    hexagons[f'{location} pipeline hydro capacity'] = hydro_capacities
    hexagons[f'{location} pipeline electrolyzer capacity'] = electrolyzer_capacities
    hexagons[f'{location} pipeline battery capacity'] = battery_capacities
    hexagons[f'{location} pipeline H2 storage capacity'] = h2_storages
    hexagons[f'{location} pipeline production cost'] = lcohs_pipeline

    print(f"Pipeline optimization for {location} completed in {pipeline_time} s")

C:\Users\lukas\AppData\Local\Temp\ipykernel_56756\897372266.py:39: FutureWarning: Non-integer 'periods' in pd.date_range, pd.timedelta_range, pd.period_range, and pd.interval_range are deprecated and will raise in a future version.
  index = pd.date_range(start_date, end_date, periods=annual_deliveries)
C:\Users\lukas\AppData\Local\Temp\ipykernel_56756\3564070593.py:79: DeprecatedWarning: lopf is deprecated as of 0.24 and will be removed in 1.0. Use linopy-based function ``n.optimize()`` instead. Migrate extra functionalities: https://pypsa.readthedocs.io/en/latest/examples/optimization-with-linopy-migrate-extra-functionalities.html.
  n.lopf(solver_name=solver,


Read LP format model from file C:\Users\lukas\AppData\Local\Temp\pypsa-problem-p1v4ova9.lp
Reading time = 0.64 seconds
obj: 219600 rows, 96633 columns, 419240 nonzeros
Total cost (objective): 38704680.34550881
Total hydrogen produced: 3809682.999999998 kg H2
Optimal wind capacity: 0.0 MW
Optimal solar capacity: 231.88970729830515 MW
Optimal hydro capacity: 0.0 MW
Optimal electrolyzer capacity: 111.87252375358388 MW
Optimal battery capacity: 0.0 MW
Optimal hydrogen storage capacity: 1129.4212524148006 MWh
Levelized Cost of Hydrogen (LCOH): 10.159554048331271 per kg H2
Pipeline optimization for Southern completed in 14.25 s


In [44]:
print(hexagons.loc[134, 'theo_turbines'])
print(hexagons.loc[134, 'theo_pv'])

38.0
262.0


In [ ]:
# Hexagon 100 - 80 years, 10%
# obj: 219600 rows, 96633 columns, 428850 nonzeros
# Total cost (objective): 743987.6139874924
# Total hydrogen produced: 140400.0 kg H2
# Optimal wind capacity: 0.0 MW
# Optimal solar capacity: 0.0 MW
# Optimal hydro capacity: 1.0994972867463215 MW
# Optimal electrolyzer capacity: 1.0673798277299282 MW
# Optimal battery capacity: 0.0 MW
# Optimal hydrogen storage capacity: 189.13614754098342 MWh
# Levelized Cost of Hydrogen (LCOH): 5.299057079682994 per kg H2

# Hexagon 100 - 80 years, 7.5%
# obj: 219600 rows, 96633 columns, 428850 nonzeros
# Total cost (objective): 680408.7007691233
# Total hydrogen produced: 140400.0 kg H2
# Optimal wind capacity: 0.0 MW
# Optimal solar capacity: 0.0 MW
# Optimal hydro capacity: 1.099497286746313 MW
# Optimal electrolyzer capacity: 1.06737982772992 MW
# Optimal battery capacity: 0.0 MW
# Optimal hydrogen storage capacity: 189.13614754097384 MWh
# Levelized Cost of Hydrogen (LCOH): 4.846215817443898 per kg H2

# Old code

In [30]:
# Cell 3: Define optimize_hydrogen_plant function
def optimize_hydrogen_plant_old(wind_potential, pv_potential, times, demand_profile,
                            wind_max_capacity, pv_max_capacity, 
                            country_series, water_limit=None):
    '''
    Optimizes the size of green hydrogen plant components based on renewable potential, hydrogen demand, and country parameters.
    '''
   
    if water_limit != None:
        total_hydrogen_demand = demand_profile['Demand'].sum()
        water_constraint = total_hydrogen_demand <= water_limit * 111.57  # kg H2 per cubic meter of water
        if not water_constraint:
            print('Not enough water to meet hydrogen demand!')
            return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan

    # Set up network
    n = pypsa.Network(override_component_attrs=aux.create_override_components())
    n.set_snapshots(times)
    n.import_from_csv_folder("Parameters/Basic_H2_plant_old")
    n.add('Load', 'Hydrogen demand', bus='Hydrogen', p_set=demand_profile['Demand'] / 1000 * 39.4)

    # Send the weather data to the model
    n.generators_t.p_max_pu['Wind'] = wind_potential
    n.generators_t.p_max_pu['Solar'] = pv_potential

    n.generators.loc['Wind', 'p_nom_max'] = wind_max_capacity * 4
    n.generators.loc['Solar', 'p_nom_max'] = pv_max_capacity

    n.generators.loc['Wind', 'capital_cost'] *= CRF(country_series['Wind interest rate'], country_series['Wind lifetime (years)'])
    n.generators.loc['Solar', 'capital_cost'] *= CRF(country_series['Solar interest rate'], country_series['Solar lifetime (years)'])
    for item in [n.links, n.stores, n.storage_units]:
        item.capital_cost *= CRF(country_series['Plant interest rate'], country_series['Plant lifetime (years)'])

    solver = 'gurobi'
    n.lopf(solver_name=solver, solver_options={'LogToConsole': 0, 'OutputFlag': 0}, pyomo=False, extra_functionality=aux.extra_functionalities)

    # Output results
    total_cost = n.objective
    total_hydrogen_produced = n.loads_t.p_set.sum().iloc[0] / 39.4 * 1000  # in kg H2

    # Print detailed breakdown of costs
    print(f"Total cost (objective): {total_cost}")
    print(f"Total hydrogen produced: {total_hydrogen_produced} kg H2")
    
    # Breakdown of installed capacities and costs
    wind_capacity = n.generators.p_nom_opt['Wind']
    solar_capacity = n.generators.p_nom_opt['Solar']
    electrolyzer_capacity = n.links.p_nom_opt['Electrolysis']
    battery_capacity = n.storage_units.p_nom_opt['Battery']
    h2_storage = n.stores.e_nom_opt['Compressed H2 Store']

    print(f"Optimal wind capacity: {wind_capacity} MW")
    print(f"Optimal solar capacity: {solar_capacity} MW")
    print(f"Optimal electrolyzer capacity: {electrolyzer_capacity} MW")
    print(f"Optimal battery capacity: {battery_capacity} MW")
    print(f"Optimal hydrogen storage capacity: {h2_storage} MWh")
    
    # Calculate and print the Levelized Cost of Hydrogen (LCOH)
    lcoh = total_cost / total_hydrogen_produced  # cost per kg H2
    print(f"Levelized Cost of Hydrogen (LCOH): {lcoh} per kg H2")
    
    return lcoh, wind_capacity, solar_capacity, electrolyzer_capacity, battery_capacity, h2_storage


In [32]:
specific_hexagon_index = 100  # Replace with the specific hexagon index or h3_index you want to process

for location in demand_centers:
    hexagon = specific_hexagon_index  # Use only the specific hexagon

    hydrogen_demand_trucking, hydrogen_demand_pipeline = demand_schedule(
        demand_parameters.loc[location, 'Annual demand [kg/a]'],
        hexagons.loc[hexagon, f'{location} trucking state'],
        transport_excel_path,
        weather_excel_path
    )
    country_series = country_parameters.loc[hexagons.country[hexagon]]
    
    lcoh, wind_capacity, solar_capacity, electrolyzer_capacity, battery_capacity, h2_storage = \
        optimize_hydrogen_plant_old(
            wind_profile.sel(hexagon=hexagon),
            pv_profile.sel(hexagon=hexagon),
            wind_profile.time,
            hydrogen_demand_trucking,
            hexagons.loc[hexagon, 'theo_turbines'],
            hexagons.loc[hexagon, 'theo_pv'],
            country_series
        )

C:\Users\lukas\AppData\Local\Temp\ipykernel_24496\2556442557.py:17: FutureWarning: Non-integer 'periods' in pd.date_range, pd.timedelta_range, pd.period_range, and pd.interval_range are deprecated and will raise in a future version.
  index = pd.date_range(start_date, end_date, periods=annual_deliveries)
C:\Users\lukas\AppData\Local\Temp\ipykernel_24496\260917163.py:35: DeprecatedWarning: lopf is deprecated as of 0.24 and will be removed in 1.0. Use linopy-based function ``n.optimize()`` instead. Migrate extra functionalities: https://pypsa.readthedocs.io/en/latest/examples/optimization-with-linopy-migrate-extra-functionalities.html.
  n.lopf(solver_name=solver, solver_options={'LogToConsole': 0, 'OutputFlag': 0}, pyomo=False, extra_functionality=aux.extra_functionalities)


Read LP format model from file C:\Users\lukas\AppData\Local\Temp\pypsa-problem-zp0318ti.lp
Reading time = 0.39 seconds
obj: 202032 rows, 87848 columns, 393714 nonzeros
Total cost (objective): 65022398.91373826
Total hydrogen produced: 5998200.000000001 kg H2
Optimal wind capacity: 0.0 MW
Optimal solar capacity: 333.58046485416946 MW
Optimal electrolyzer capacity: 100.26049395506988 MW
Optimal battery capacity: 378.43956426315555 MW
Optimal hydrogen storage capacity: 5118.436997725744 MWh
Levelized Cost of Hydrogen (LCOH): 10.840318581197401 per kg H2
